In [1]:
import torch as t

# 归并操作
**简单的记忆方法**  
假设输入的形状是(m,n,k):
* 如果指定 dim=0, 那么输出的形状就是(1,n,k)或(n,k);
* 如果指定 dim=1，那么输出的形状就是(m,1,k)或(m,k);
* 如果指定 dim=2，那么输出的形状就是(m,n,1)或(m,n).
size中是否有1，取决于参数`keepdim`,如果`keepdim=True`就会保留维度1.

In [2]:
b = t.ones(2,3)
b.sum(dim=0, keepdim=True) # shape(1,3)

tensor([[2., 2., 2.]])

In [3]:
# when keepdim=False
b.sum(dim=0, keepdim=False) # size: [3] is a vector

tensor([2., 2., 2.])

In [4]:
b.sum(dim=1) # which means that keepdim=False in default

tensor([3., 3.])

In [5]:
# exception: 行累加
a = t.arange(0, 6).view(2,3)
print(a)

tensor([[0, 1, 2],
        [3, 4, 5]])


In [6]:
a.cumsum(dim=1) # 沿着行累加

tensor([[ 0,  1,  3],
        [ 3,  7, 12]])

# 比较操作
max和min这两个操作比较特殊：
* t.max(tensor): 返回tensor中最大的一个数
* t.max(tensor,dim): 制定维上最大的数，返回tensor和下标
* t.max(tensor1,tensor2):比较两个tensor中比较大的元素  

其中，比较一个tensor和一个数，可以用clamp函数。

In [8]:
a = t.linspace(0, 15, 6).view(2,3)
a

tensor([[ 0.,  3.,  6.],
        [ 9., 12., 15.]])

In [9]:
b = t.linspace(15, 0, 6).view(2,3)
b

tensor([[15., 12.,  9.],
        [ 6.,  3.,  0.]])

In [10]:
a > b # compare tensor a and b and return bool

tensor([[0, 0, 0],
        [1, 1, 1]], dtype=torch.uint8)

In [11]:
a[a>b] # return the elements in a which is bigger than b

tensor([ 9., 12., 15.])

In [12]:
t.max(a) # find the biggest element in a

tensor(15.)

In [13]:
t.max(b, dim=1) # the first tuple is the biggest element in each row, the second means the position of these two elements

(tensor([15.,  6.]), tensor([0, 0]))

In [14]:
t.max(a,b)

tensor([[15., 12.,  9.],
        [ 9., 12., 15.]])

In [15]:
# 比较a和10较大的元素
t.clamp(a, min=10)

tensor([[10., 10., 10.],
        [10., 12., 15.]])

# Tensor和Numpy
* tensor和numpy共享内存
* 当tensor遇到不支持的操作时，可以先转为numpy，处理后再转换为tensor，转换开销很小

In [16]:
import numpy as np
a = np.ones([2, 3], dtype=np.float32)
a

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [17]:
b = t.from_numpy(a)
b

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [19]:
b = t.Tensor(a) # 可以直接把numpy对象传入Tensor,如果Numpy类不是float32的话，会新建内存
b

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [20]:
a[0,1] = 100
b

tensor([[  1., 100.,   1.],
        [  1.,   1.,   1.]])

In [21]:
c = b.numpy() # a, b, c三个共享内存
c

array([[  1., 100.,   1.],
       [  1.,   1.,   1.]], dtype=float32)

## broadcast广播
* unsqueeze或者view：为数据某一维补1

In [23]:
a = t.ones(3,2)
b = t.zeros(2, 3, 1)
print(a)
print(b)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[[0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.]]])


In [24]:
# 自动广播
a + b

tensor([[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]]])

手动广播步骤：
1. a是二维，b是三维，现在a前面补1::a.unsqueeze(0), a的形状由(3,2)变成(1,3,2)
2. a和b的形状在第一维和第三维不一样，我觉得第一步就够了，会自动广播了..

In [25]:
a.unsqueeze(0) + b

tensor([[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]]])

# 内部结构
不同tensor的头信息一般不同，但却可能使用相同的storage

In [26]:
a = t.arange(0, 6)
a.storage()

 0
 1
 2
 3
 4
 5
[torch.LongStorage of size 6]

In [28]:
b = a.view(2,3)
b.storage()

 0
 1
 2
 3
 4
 5
[torch.LongStorage of size 6]

In [29]:
# 一个对象的id可以看做它在内存中的地址
id(b.storage()) == id(a.storage())

True

In [30]:
# a change , b change too cause' they share the same storage
a[1]= 100
b

tensor([[  0, 100,   2],
        [  3,   4,   5]])

In [31]:
c = a[2:] 
c.storage()

 0
 100
 2
 3
 4
 5
[torch.LongStorage of size 6]

In [33]:
c.data_ptr(), a.data_ptr()
# 可以看到地址相差了16，因为这里相差两个元素，我的机器是64位的，每个元素占8个字节(可能)

(94622222165600, 94622222165584)

In [34]:
c[0] = -100 # c[0]的内存地址对应a[2]的内存地址
a

tensor([   0,  100, -100,    3,    4,    5])

# 其他有关Tensor的话题
## 持久化

In [38]:
if t.cuda.is_available():
    a = a.cuda(1) # 把a转为GPU1上的tensor
    t.save(a, 'a.pth')
    # 将存储在GPU1上的a加载为b
    b = t.load('a.pth') 
    #加载为c，存储在cpu
    c = t.load('a.pth', map_location=lambda storage, loc: storage)
    # 加载为d，存储在GPU0上
    d = t.load('a.pth', map_location={'cuda:1': 'cuda:0'})

RuntimeError: CUDA error: invalid device ordinal

## 向量化
比较一下向量化和非向量化的计算效率

In [39]:
def for_loop_add(x, y):
    result = []
    for i, j in zip(x, y): # zip() 将向量打包为元组
        result.append(i + j)
    return t.Tensor(result)

In [40]:
x = t.zeros(100)
y = t.ones(100)
%timeit -n 10 for_loop_add(x, y)
%timeit -n 10 x + y

707 µs ± 151 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
The slowest run took 7.79 times longer than the fastest. This could mean that an intermediate result is being cached.
5.44 µs ± 6.55 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# autograd
## Variable
> Variable封装了tensor
> 主要包含三个属性：

`data`,`grad`,`grad_fn`

In [41]:
from __future__ import print_function
import torch as t
from torch.autograd import Variable as V


In [43]:
# 从tensor中创建variable，指定需要求导
a = V(t.ones(3,4), requires_grad=True)
a

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)

In [44]:
b = V(t.zeros(3,4))
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [45]:
# variable 的是函数使用和tensor一致
c = a.add(b)  # equals to c = a + b
c 

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)

In [48]:
d = c.sum()
d.backward() # 反向传播

In [51]:
# note:注意以下两者区别：
# 前者从variable从取data出来是tensor，然后使用sum得到的是float
# 后者直接sum得到的结果类型还是variable
c.data.sum()
c.sum()

tensor(12., grad_fn=<SumBackward0>)

In [52]:
a.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [55]:
# 此处虽然没有指定c需要求导，但是c依赖于a，a需要求导，所以c的requires_grad会自动设置为True
print(a.requires_grad)
print(b.requires_grad)
print(c.requires_grad)

True
False
True


In [56]:
# 由用户创建的Variable属于叶子节点
print(a.is_leaf)
print(b.is_leaf)
print(c.is_leaf)

True
True
False


### 对比autograd计算的导数和我们手动计算的区别：
eg.  
$$y = x^2e^x$$
导函数为：  
$$\frac {\partial y} {\partial x}  = 2xe^x + x^2e^x$$

In [57]:
def f(x):
    y = x**2 * t.exp(x)
    return y

def gradf(x):
    dx = 2*x*t.exp(x) + x**2*t.exp(x)
    return dx

In [58]:
x = V(t.randn(3,4), requires_grad=True)
y = f(x)
y

tensor([[0.8825, 0.4997, 0.4772, 0.5244],
        [0.4703, 0.0163, 0.0238, 1.4653],
        [0.0224, 0.0257, 0.0062, 0.7099]], grad_fn=<MulBackward0>)

In [59]:
y.backward(t.ones(y.size()))
x.grad

tensor([[ 3.5109, -0.1728, -0.2188,  2.4312],
        [-0.2314,  0.2874, -0.2600,  5.0917],
        [-0.2534, -0.2680,  0.1701,  3.0057]])

In [60]:
gradf(x)

tensor([[ 3.5109, -0.1728, -0.2188,  2.4312],
        [-0.2314,  0.2874, -0.2600,  5.0917],
        [-0.2534, -0.2680,  0.1701,  3.0057]], grad_fn=<AddBackward0>)